# Final Distribution

In [1]:
from config import PROJECT_ID, INITIAL_TS, SNAPSHOT_TS, \
    GENESIS_AND_ETH2_TYPE, GENESIS_AND_ETH2_DATASET_NAME, GENESIS_AND_ETH2_DISTRIBUTION_TABLE_NAME, \
    GAS_ANALYSIS_DATASET_NAME, GAS_ANALYSIS_DISTRIBUTION_TABLE_NAME, GAS_ANALYSIS_TYPE, \
    ETH_ANALYSIS_TYPE, ETH_ANALYSIS_DATASET_NAME, ETH_ANALYSIS_DISTRIBUTION_TABLE_NAME, \
    ERC721_ANALYSIS_TYPE, ERC721_ANALYSIS_DATASET_NAME, ERC721_ANALYSIS_DISTRIBUTION_TABLE_NAME, \
    ERC20_ANALYSIS_TYPE, ERC20_ANALYSIS_DATASET_NAME, ERC20_ANALYSIS_DISTRIBUTION_TABLE_NAME, \
    ERC20_TOP_TYPE, ERC20_TOP_DISTRIBUTION_TABLE_NAME, \
    FINAL_DATASET_NAME, FINAL_DISTRIBUTION_VIEW_NAME, CONTRACT_ADDRESSES_TABLE, GIFT_PER_ADDRESS_PIVOT_VIEW_NAME
from src.utils import create_view, create_table, drop_table

DROP_TABLES = True
CREATE_TABLES = True
DROP_VIEW = True
CREATE_VIEW = True

## Set cyberGift for the Audiences

In [ ]:
gift_per_audience = {
    'Average Citizens. ETH Analysis' : 40e13,
    'Passionate Investors. ERC20 Analysis': 5e13,
    'Masters of the Great Web. ERC721 Analysis': 5e13,
    'Extraordinary Hackers. Gas Analysis': 5e13,
    'True Heroes of the Great Web. Genesis and ETH2 Stakers': 5e13,
    'Key Opinion Leaders. Top Token Holders': 5e13
}

## Create Contracts Table

In [2]:
query_1 = f'''
    SELECT
        to_address as address
    FROM `bigquery-public-data.crypto_ethereum.traces`
    WHERE block_timestamp >= '{INITIAL_TS}'
      AND block_timestamp <= '{SNAPSHOT_TS}'
      AND trace_type ='create'
      AND status = 1
'''

if DROP_TABLES:
    drop_table(CONTRACT_ADDRESSES_TABLE, dataset_name=FINAL_DATASET_NAME)
if CREATE_TABLES:
    create_table(query_1, CONTRACT_ADDRESSES_TABLE, dataset_name=FINAL_DATASET_NAME)

Table final:contract_addresses has not been deleted.
404 DELETE https://bigquery.googleapis.com/bigquery/v2/projects/cosmic-keep-223223/datasets/final/tables/contract_addresses?prettyPrint=false: Not found: Table cosmic-keep-223223:final.contract_addresses
Table final:contract_addresses has been created and filled 38527292 rows.


## Create View

In [4]:
query_1 = f'''
    SELECT
        type,
        subtype,
        address,
        grade
    FROM (
        SELECT
            '{GENESIS_AND_ETH2_TYPE}' AS type,
            subtype,
            address,
            grade
        FROM `{PROJECT_ID}.{GENESIS_AND_ETH2_DATASET_NAME}.{GENESIS_AND_ETH2_DISTRIBUTION_TABLE_NAME}`
        WHERE grade IS NOT NULL

        UNION ALL

        SELECT
            '{GAS_ANALYSIS_TYPE}' AS type,
            subtype,
            address,
            grade
        FROM `{PROJECT_ID}.{GAS_ANALYSIS_DATASET_NAME}.{GAS_ANALYSIS_DISTRIBUTION_TABLE_NAME}`
        WHERE grade IS NOT NULL

        UNION ALL

        SELECT
            '{ETH_ANALYSIS_TYPE}' AS type,
            subtype,
            address,
            grade
        FROM `{PROJECT_ID}.{ETH_ANALYSIS_DATASET_NAME}.{ETH_ANALYSIS_DISTRIBUTION_TABLE_NAME}`
        WHERE grade IS NOT NULL

        UNION ALL

        SELECT
            '{ERC721_ANALYSIS_TYPE}' AS type,
            '' AS subtype,
            address,
            grade
        FROM `{PROJECT_ID}.{ERC721_ANALYSIS_DATASET_NAME}.{ERC721_ANALYSIS_DISTRIBUTION_TABLE_NAME}`
        WHERE grade IS NOT NULL

        UNION ALL

        SELECT
            '{ERC20_ANALYSIS_TYPE}' AS type,
            '' AS subtype,
            address,
            grade
        FROM `{PROJECT_ID}.{ERC20_ANALYSIS_DATASET_NAME}.{ERC20_ANALYSIS_DISTRIBUTION_TABLE_NAME}`
        WHERE grade IS NOT NULL

        UNION ALL

        SELECT
            '{ERC20_TOP_TYPE}' AS type,
            '' AS subtype,
            address,
            grade
        FROM `{PROJECT_ID}.{ERC20_ANALYSIS_DATASET_NAME}.{ERC20_TOP_DISTRIBUTION_TABLE_NAME}`
        WHERE grade IS NOT NULL)
    WHERE address NOT IN (SELECT address FROM `{PROJECT_ID}.{FINAL_DATASET_NAME}.{CONTRACT_ADDRESSES_TABLE}`)
'''

if DROP_VIEW:
    drop_table(FINAL_DISTRIBUTION_VIEW_NAME, dataset_name=FINAL_DATASET_NAME)

if CREATE_VIEW:
    create_view(query_1, FINAL_DISTRIBUTION_VIEW_NAME)

Table final:final_distribution has not been deleted.
404 DELETE https://bigquery.googleapis.com/bigquery/v2/projects/cosmic-keep-223223/datasets/final/tables/final_distribution?prettyPrint=false: Not found: Table cosmic-keep-223223:final.final_distribution
View VIEW:cosmic-keep-223223.final.final_distribution has been created.


In [5]:
query_2 = f'''
    SELECT
        type,
        grade_1 as `Number_of_addresses_in_Grade_1`,
        ROUND(IF(grade_1 > 0, gift_amount_per_type / grade_1 / number_of_grades / 1e6, 0), 2) as `Gift_per_address_for_Grade_1`,
        grade_2 as `Number_of_addresses_in_Grade_2`,
        ROUND(IF(grade_2 > 0, gift_amount_per_type / grade_2 / number_of_grades / 1e6, 0), 2) as `Gift_per_address_for_Grade_2`,
        grade_3 as `Number_of_addresses_in_Grade_3`,
        ROUND(IF(grade_3 > 0, gift_amount_per_type / grade_3 / number_of_grades / 1e6, 0), 2) as `Gift_per_address_for_Grade_3`,
        total_addresses as `Number_of_addresses_in_type`,
        gift_amount_per_type as `Gift_amount_per_type`
    FROM (
        SELECT
            type,
            SUM(IF(grade=1, address_count, 0)) as grade_1,
            SUM(IF(grade=2, address_count, 0)) as grade_2,
            SUM(IF(grade=3, address_count, 0)) as grade_3,
            SUM(address_count) as total_addresses,
            COUNTIF(address_count > 0) as number_of_grades
        FROM (
            SELECT
                type,
                grade,
                count(distinct address) as address_count
            FROM `{PROJECT_ID}.{FINAL_DATASET_NAME}.{FINAL_DISTRIBUTION_VIEW_NAME}`
            WHERE grade IN (1, 2, 3)
            GROUP BY type, grade)
        GROUP BY type)
    LEFT JOIN (
        SELECT
            gift.type as type,
            gift.amount_per_type as gift_amount_per_type
        FROM UNNEST([{''.join(f"STRUCT('{k}' AS type, {v} AS amount_per_type), " for k,v in gift_per_audience.items())[:-2]}]) as gift
        )
    USING (type)
'''

if DROP_VIEW:
    drop_table(GIFT_PER_ADDRESS_PIVOT_VIEW_NAME, dataset_name=FINAL_DATASET_NAME)

if CREATE_VIEW:
    create_view(query_2, GIFT_PER_ADDRESS_PIVOT_VIEW_NAME)

Table final:gift_per_address_pivot has not been deleted.
404 DELETE https://bigquery.googleapis.com/bigquery/v2/projects/cosmic-keep-223223/datasets/final/tables/gift_per_address_pivot?prettyPrint=false: Not found: Table cosmic-keep-223223:final.gift_per_address_pivot
View VIEW:cosmic-keep-223223.final.gift_per_address_pivot has been created.
